## Setup

In [1]:
import urllib.request
import zipfile
import pandas as pd
from urllib.parse import urlencode
import json
import gzip

### Define Data

In [2]:
DATA_DIR = './data/'
ZIPS_URL = 'http://www2.census.gov/geo/docs/maps-data/data/gazetteer/2017_Gazetteer/2017_Gaz_zcta_national.zip'
ZIPS_FP = DATA_DIR + 'zips-2017.zip'
ZHVI_URL = 'http://files.zillowstatic.com/research/public/Zip/Zip_Zhvi_SingleFamilyResidence.csv'
ZHVI_FP = DATA_DIR + 'sfr.csv'

### (Optional) Refresh Data

In [ ]:
urllib.request.urlretrieve(ZIPS_URL, ZIPS_FP);

In [ ]:
urllib.request.urlretrieve(ZHVI_URL, ZHVI_FP);

## Home Prices

In [3]:
df = pd.read_csv(ZHVI_FP)
counties = ['San Francisco', 'San Mateo', 'San Jose']
df = df[df['CountyName'].isin(counties)]
zips = df['RegionName']

## Schools

In [4]:
df2 = pd.read_csv(ZIPS_FP, sep='\t')
df2 = df2[df2['GEOID'].isin(zips.values)]
df2.head()

,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
30954,94002,14661770,22417,5.661,0.009,37.514354,-122.298901
30955,94005,11384557,447655,4.396,0.173,37.688826,-122.408935
30956,94010,30667758,226705,11.841,0.088,37.570280,-122.365778
30957,94014,16319097,0,6.301,0.000,37.690884,-122.447441
30958,94015,14894690,0,5.751,0.000,37.681312,-122.480634


In [ ]:
base = 'https://www.greatschools.org/search/search.page?'
params = {
    'lat': '37.469453',
    'lon': '-122.411841',
    'zip': '94010',
    'state': 'CA'
}
url = base + urlencode(params)
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:58.0) Gecko/20100101 Firefox/58.0",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "accept-encoding": "gzip,deflate,br",
    "accept-language": "en-US,en;q=0.5",
}

In [ ]:
req = urllib.request.Request(url, None, headers)
with urllib.request.urlopen(req) as res:
    html = gzip.decompress(res.read()).decode('utf-8')

In [ ]:
raw = html.find('gon.map_points')
start = html.find('[',raw)
end = html.find(']',raw)
data = html[start:end+1]
schools = json.loads(data)
rated = [i for i in schools if i['gsRating'] > 0]

In [ ]:
rated

In [ ]:
# Calculating the average rating for each zipcode
total_rating = 0
num_schools = len(rated)
total_schools.append(num_schools)
for school in rated:
    rating = school['gsRating']
    total_rating = total_rating + rating
avg_rating = round(total_rating/num_schools, 2)
print(avg_rating)

# Creating a list of all types of grade ranges
grades = []
grade_range= []
for school in rated:
    grade = school['gradeRange']
    grades.append(grade)
[grade_range.append(item) for item in grades if item not in grade_range]
grade_range = [grade_range]
print(grade_range)
# Creating a final table
d = {'zipcode': zipcodes, 'avg_rating': avg_ratings, 
     'number_of_schools': total_schools, 'grade_ranges': [grade_range]}
schools_df = pd.DataFrame(data =d)
schools_df = schools_df.set_index('zipcode')
schools_df.head()

In [5]:
# Start here -> Cleaning up data
df2 = df2.drop(["ALAND", "AWATER", "ALAND_SQMI", "AWATER_SQMI"], axis =1)
df2.columns=["zipcode", "lat", "long"]
df2 = df2[df2.zipcode != 94020]
df2.head()

,zipcode,lat,long
30954,94002,37.514354,-122.298901
30955,94005,37.688826,-122.408935
30956,94010,37.570280,-122.365778
30957,94014,37.690884,-122.447441
30958,94015,37.681312,-122.480634


In [6]:
# Creating empty lists to store the data
avg_ratings = []
grade_ranges = []
total_schools = []
zipcodes = df2['zipcode'].tolist()


# Iterate through all the zipcodes & grab data
for index, row in df2.iterrows():
    base = 'https://www.greatschools.org/search/search.page?'
    params = {
    'lat': row['lat'],
    'lon': row['long'],
    'zip': row['zipcode'],
    'state': 'CA',
    }
    url = base + urlencode(params)
    
    headers = {
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:58.0) Gecko/20100101 Firefox/58.0",
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "accept-encoding": "gzip,deflate,br",
        "accept-language": "en-US,en;q=0.5",
    }
    
    req = urllib.request.Request(url, None, headers)
    with urllib.request.urlopen(req) as res:
        html = gzip.decompress(res.read()).decode('utf-8')
    
    raw = html.find('gon.map_points')
    start = html.find('[',raw)
    end = html.find(']',raw)
    data = html[start:end+1]
    schools = json.loads(data)
    rated = [i for i in schools if i['gsRating'] > 0]
    
    # Calculating the total number of schools & average rating for each zipcode
    total_rating = 0
    num_schools = len(rated)
    total_schools.append(num_schools)
    for school in rated:
        rating = school['gsRating']
        total_rating = total_rating + rating
    avg_rating = round(total_rating/num_schools, 2)
    avg_ratings.append(avg_rating)

    # Creating a list of all types of grade ranges
    grades = []
    grade_range= []
    for school in rated:
        grade = school['gradeRange']
        grades.append(grade)
    [grade_range.append(item) for item in grades if item not in grade_range]
    grade_ranges.append(grade_range)
    

In [7]:
# Creating a final table
d = {'zipcode': zipcodes, 'avg_rating': avg_ratings, 
     'number_of_schools': total_schools, 'grade_ranges': grade_ranges}
schools_df = pd.DataFrame(data =d)
schools_df = schools_df.set_index('zipcode')
schools_df = schools_df.sort_values('avg_rating', ascending = False)
schools_df

,avg_rating,grade_ranges,number_of_schools
zipcode,,,
94028,8.12,"[K-5, 9-12, 4-8, K-3, K-8, K-6, 6-8]",16
94403,7.28,"[9-12, K-5, 6-8, K-4, 5-8, 1-5, K-8, K-6, K-12]",47
94402,7.19,"[9-12, K-5, 6-8, K-4, 5-8, 1-5, K-8, K-6, K-12]",42
94401,7.18,"[9-12, K-5, 6-8, 1-5, K-6, K-8, K-12]",44
94404,7.17,"[9-12, K-5, K-4, 5-8, 6-8, 1-5, K-8, K-6, K-12]",41
94030,6.95,"[9-12, K-5, 6-8, K-8, 7-9]",43
94002,6.92,"[3-8, K-5, K-4, 5-8, 9-12, 6-8, 1-5, K-8, K-6,...",49
94010,6.76,"[9-12, K-5, 6-8, 1-5, K-8, 7-9]",46
94131,6.58,"[9-12, K-8, K-5, 6-8, 5-8, 1-5, K-6, 9-12 & un...",100
